In [70]:
import requests
import pandas as pd

In [71]:
from_easting = 7.42354
from_northing = 46.93682

to_easting = 7.57965
to_northing = 46.94323

# Car

In [72]:
url_car = rf"http://127.0.0.1:5000/route/v1/driving/{from_easting},{from_northing};{to_easting},{to_northing}?steps=true"

In [73]:
%%time
res_car = requests.get(url_car).json()

CPU times: total: 0 ns
Wall time: 29.5 ms


In [75]:
res_car['routes'][0]['distance'] / 1000

17.196099999999998

In [76]:
res_car['routes'][0]['duration'] / 60

23.066666666666666

# Bicycle

In [77]:
url_bicycle = rf"http://127.0.0.1:6000/route/v1/driving/{from_easting},{from_northing};{to_easting},{to_northing}?steps=true"

In [78]:
%%time
res_bicycle = requests.get(url_bicycle).json()

CPU times: total: 0 ns
Wall time: 20 ms


In [79]:
res_bicycle['routes'][0]['distance'] / 1000

14.405299999999999

In [80]:
res_bicycle['routes'][0]['duration'] / 60

62.50833333333333

# Foot

In [81]:
url_foot = rf"http://127.0.0.1:7000/route/v1/driving/{from_easting},{from_northing};{to_easting},{to_northing}?steps=true"

In [82]:
res_foot = requests.get(url_foot).json()

In [83]:
res_foot['routes'][0]['distance'] / 1000

14.326600000000001

In [84]:
res_foot['routes'][0]['duration'] / 60

172.07166666666666

# Matrix routing

In [104]:
def get_matrix(port):
    url_matrix = rf"http://127.0.0.1:{port}/table/v1/driving/8.780822753906252,47.441092383072515;7.10161,46.80693;7.94136,47.2878;7.94422,47.28723;7.94426,47.2943;8.68576,47.369858;8.71032,47.34795;8.71756,47.34778;8.715572,47.352184;8.71318,47.35402;8.71958,47.3484;8.72482,47.34503;8.72599,47.3525;8.68604,47.42489;8.68596,47.42726;8.68952,47.42656;8.72308,47.40897;8.52907,47.3799;8.531334,47.383064;8.53395,47.3783;8.5332,47.38517;8.53585,47.38198;8.54235,47.38068;8.54407,47.3797;8.54285,47.38186;8.54211,47.3832;8.54352,47.38557;8.548451,47.406967;8.54569,47.3793;8.547121,47.37884;8.54875,47.37457;8.55008,47.3723;8.54627,47.38178;8.548753,47.380665;8.5489,47.38306;8.61861,47.41944;8.562324,47.449635;8.59262,47.412;6.94045,46.82023;8.62862,47.42056;8.595176,47.39685;8.60413,47.40381;8.751117,47.384808;8.752607,47.38695;8.71293,47.49962;8.71683,47.50253;8.72369,47.50053;8.72231,47.50538;8.72273,47.4985;8.72848,47.5046;8.73156,47.4943;8.73178,47.49617;8.738186,47.497242;8.73689,47.50266;8.73392,47.5014;8.74231,47.49312;8.74411,47.49621;8.7668,47.48703;7.14432,46.8013;7.15202,46.7991;7.15019,46.8015;7.15354,46.80576;7.161402,46.803467;7.1611,46.80615;8.77705,47.37055;8.78385,47.3672;8.62663,47.43851;8.65838,47.38477?destinations=0&annotations=duration,distance"
    res_matrix = requests.get(url_matrix).json()
    return res_matrix

In [113]:
%%time
res_car = get_matrix(5000)

CPU times: total: 15.6 ms
Wall time: 54 ms


In [114]:
%%time
res_bicycle = get_matrix(6000)

CPU times: total: 15.6 ms
Wall time: 179 ms


In [115]:
%%time
res_foot = get_matrix(7000)

CPU times: total: 0 ns
Wall time: 151 ms


In [136]:
df_durations = pd.DataFrame(list(zip(*[[x[0] for x in res_car["durations"]], [x[0] for x in res_bicycle["durations"]], [x[0] for x in res_foot["durations"]]])), columns=["car", "bicycle", "foot"]) / 60 / 60
print(len(df_durations))

68


In [133]:
df_durations

,car,bicycle,foot
0,0.000000,0.000000,0.000000
1,2.168583,11.990167,33.468806
2,1.077139,5.693306,15.478972
3,1.071806,5.683083,15.443667
4,1.074028,5.625944,15.325000
...,...,...,...
63,2.138639,11.640556,32.490222
64,0.208389,0.700306,1.941667
65,0.216278,0.697778,1.972889
66,0.367806,1.121889,2.936028


In [139]:
df_distances = pd.DataFrame(list(zip(*[[x[0] for x in res_car["distances"]], [x[0] for x in res_bicycle["distances"]], [x[0] for x in res_foot["distances"]]])), columns=["car", "bicycle", "foot"]) / 1000

In [140]:
df_distances
print(len(df_distances))

68


In [141]:
df_distances

,car,bicycle,foot
0,0.0000,0.0000,0.0000
1,181.0175,173.0222,167.9472
2,83.4944,80.0349,76.9700
3,83.3962,79.9368,76.5318
4,83.3952,79.0578,76.2001
...,...,...,...
63,176.1773,167.9385,163.0881
64,9.9576,10.0945,9.6363
65,10.1832,10.1678,9.7924
66,16.7395,15.3610,14.6996
